In [1]:
import os
import pandas as pd
from pathlib import Path
from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt
import datetime

converting  parquet files to csv files :

In [ ]:
base_path = Path("../Input/Tennis Schema/data")


folders_to_read = [ "raw_odds_parquet",
                    "raw_point_by_point_parquet","raw_statistics_parquet", 
                    "raw_tennis_power_parquet", "raw_votes_parquet"]  

final_data = {}

for folder_name in folders_to_read:
    all_parquet_files = list(base_path.glob(f"*/{folder_name}/*.parquet"))
    
    dfs = []
    for pq_file in all_parquet_files:
        df = pd.read_parquet(pq_file,engine='fastparquet')
        dfs.append(df)

    
    combined_df = pd.concat(dfs, ignore_index=True)
    final_data[folder_name] = combined_df

match_folder = Path("../Input/Tennis Schema/data")

all_files = list(match_folder.glob("*/raw_match_parquet/*.parquet"))

grouped_files = defaultdict(list)

for f in all_files:
    parts = f.stem.split("_")
    base_parts = []
    for p in parts:
        if p.isdigit():
            break
        base_parts.append(p)
    base_name = "_".join(base_parts)
    grouped_files[base_name].append(f)

match_dfs = {
    group: pd.concat([pd.read_parquet(f, engine='fastparquet') for f in files], ignore_index=True)
    for group, files in grouped_files.items()
}

for name, df in match_dfs.items():
    df.to_csv(f"{name}.csv", index=False)